In [3]:
import pickle as pickle

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.preprocessing as preprocessing
import sklearn.neural_network as nn
import sklearn.svm as svm
import sklearn.metrics as metrics

In [4]:
def load_dataset(n):
    data = pd.read_csv(f'Datasets/Duplicated{n}MeanPastMatches.csv')
    diff_data = pd.DataFrame()
    # Calculate diffs
    cols = ("age", "ht", "ace", "df", "svpt", "1stIn", "1stWon", "2ndWon", "SvGms", "bpSaved", "bpFaced", "rank")
    for col in cols:
        diff_data[f'{n}_diff_{col}'] = data['player1_' + col] - data['player2_' + col]

    return diff_data

ns = range(1, 16)
datasets = []
for n in ns:
    datasets.append(load_dataset(n))

data = pd.concat(datasets, axis=1)
quant_cols = data.columns

first_dataset = pd.read_csv('Datasets/Duplicated1MeanPastMatches.csv')
data['date'] = first_dataset['tourney_date']

ohc = preprocessing.OneHotEncoder()
surfaces = ohc.fit_transform(first_dataset['surface'].values.reshape((-1, 1))).toarray()
surface_df = pd.DataFrame(surfaces, columns=['surface' + '_' + str(i) for i in range(surfaces.shape[1])])
data = pd.concat([data, surface_df], axis=1)
data['same_hand'] = (first_dataset['player1_hand'] == first_dataset['player2_hand']).astype(int)
data['label'] = first_dataset['label']

scaler = preprocessing.StandardScaler()
data[quant_cols] = scaler.fit_transform(data[quant_cols])
data = data.dropna()

data_train = data[data['date'] < 20150101].drop('date', axis=1)
data_val = data[(data['date'] >= 20150101) & (data['date'] < 20190101)].drop('date', axis=1)
data_test = data[data['date'] >= 20190101].drop('date', axis=1)


X_train = data_train.drop('label', axis=1)
y_train = data_train['label']

X_val = data_val.drop('label', axis=1)
y_val = data_val['label']

X_test = data_test.drop('label', axis=1)
y_test = data_test['label']

In [6]:
# Load models

svm = pickle.load(open('./models/Full/concatenados/svm_model.pkl', 'rb'))
svm_linear = pickle.load(open('./models/Full/concatenados/linear_svm.pkl', 'rb'))
mlp = pickle.load(open('./models/Full/concatenados/mlp_model.pkl', 'rb'))

/home/aurmeneta/IIC2433/ProyectoMineriaDeDatos/.venv/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [35]:
X = data_test.sample()
x = X.drop('label', axis=1)
y = X['label']

mlp.predict(x) == y, svm.predict(x) == y, svm_linear.predict(x) == y

(137420    True
 Name: label, dtype: bool,
 137420    False
 Name: label, dtype: bool,
 137420    True
 Name: label, dtype: bool)